In [1]:
# conda activate covid19
import numpy as np
import pandas as pd
np.__version__, pd.__version__

('1.19.2', '1.2.2')

1) Load the raw csv file into a pandas dataframe

In [2]:
# https://data.ontario.ca/dataset/confirmed-positive-cases-of-covid-19-in-ontario
csvName = 'data/conposcovidloc-2021-02-22'
rawDf = pd.read_csv(csvName + '.csv', 
                  header=0, 
                  index_col='Row_ID', 
                  parse_dates=['Accurate_Episode_Date','Case_Reported_Date','Test_Reported_Date','Specimen_Date'])
rawDf.head()

,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
Row_ID,,,,,,,,,,,,,,,,,
1,2020-04-07,2020-04-24,2020-04-24,2020-04-22,40s,FEMALE,OB,Resolved,Yes,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
2,2020-05-29,2020-06-05,2020-06-05,2020-06-05,50s,FEMALE,OB,Resolved,Yes,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
3,2020-03-26,2020-03-30,2020-03-30,2020-03-26,40s,MALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
4,2020-05-17,2020-05-18,2020-05-18,2020-05-17,40s,FEMALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
5,2020-04-05,2020-04-11,2020-04-11,2020-04-10,40s,FEMALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893


In [3]:
rawDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294144 entries, 1 to 294144
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Accurate_Episode_Date      294144 non-null  datetime64[ns]
 1   Case_Reported_Date         294144 non-null  datetime64[ns]
 2   Test_Reported_Date         283613 non-null  datetime64[ns]
 3   Specimen_Date              292320 non-null  datetime64[ns]
 4   Age_Group                  294144 non-null  object        
 5   Client_Gender              294144 non-null  object        
 6   Case_AcquisitionInfo       294144 non-null  object        
 7   Outcome1                   294144 non-null  object        
 8   Outbreak_Related           65050 non-null   object        
 9   Reporting_PHU_ID           294144 non-null  int64         
 10  Reporting_PHU              294144 non-null  object        
 11  Reporting_PHU_Address      294144 non-null  object  

In [4]:
# https://www.geeksforgeeks.org/creating-a-sqlite-database-from-csv-with-python/

# Import required libraries 
import sqlite3 
import pandas as pd 

# Connect to SQLite database 
conn = sqlite3.connect(r'conposcovidloc.db') 

# Write the data to a sqlite table 
rawDf.to_sql(csvName, conn, if_exists='replace', index=True) 

# Close connection to SQLite database 
conn.close() 

2) Extract the Public Health Unit data into a csv file

In [5]:
phuIndexColumn = 'Reporting_PHU_ID'
phuInfoColumns = ['Reporting_PHU', 'Reporting_PHU_Address','Reporting_PHU_City','Reporting_PHU_Postal_Code','Reporting_PHU_Website','Reporting_PHU_Latitude','Reporting_PHU_Longitude']
phuAllColumns =  [phuIndexColumn] + phuInfoColumns
phuDf = rawDf[phuAllColumns].set_index(phuIndexColumn).drop_duplicates()
phuDf.to_csv('PublicHealthUnits.csv')
phuDf.head()

,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
Reporting_PHU_ID,,,,,,,
2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
2236,Halton Region Health Department,1151 Bronte Road,Oakville,L6M 3Ll,www.halton.ca/For-Residents/Public-Health/,43.413997,-79.744796
2266,Wellington-Dufferin-Guelph Public Health,160 Chancellors Way,Guelph,N1G 0E1,www.wdgpublichealth.ca,43.524881,-80.233743
2241,"Kingston, Frontenac and Lennox & Addington Pub...",221 Portsmouth Avenue,Kingston,K7M 1V5,www.kflaph.ca,44.227874,-76.525211
3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358


In [6]:
phuDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 2253 to 2243
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Reporting_PHU              34 non-null     object 
 1   Reporting_PHU_Address      34 non-null     object 
 2   Reporting_PHU_City         34 non-null     object 
 3   Reporting_PHU_Postal_Code  34 non-null     object 
 4   Reporting_PHU_Website      34 non-null     object 
 5   Reporting_PHU_Latitude     34 non-null     float64
 6   Reporting_PHU_Longitude    34 non-null     float64
dtypes: float64(2), object(5)
memory usage: 2.1+ KB


3) Drop the Public Health Unit Information columns from the raw data frame into a new caseDf dataframe.

In [7]:
caseDf = rawDf.drop(phuInfoColumns, axis = 1)
caseDf['Outbreak_Related'] = caseDf['Outbreak_Related'].fillna('No')
caseDf.head()

,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID
Row_ID,,,,,,,,,,
1,2020-04-07,2020-04-24,2020-04-24,2020-04-22,40s,FEMALE,OB,Resolved,Yes,2253
2,2020-05-29,2020-06-05,2020-06-05,2020-06-05,50s,FEMALE,OB,Resolved,Yes,2253
3,2020-03-26,2020-03-30,2020-03-30,2020-03-26,40s,MALE,CC,Resolved,No,2253
4,2020-05-17,2020-05-18,2020-05-18,2020-05-17,40s,FEMALE,CC,Resolved,No,2253
5,2020-04-05,2020-04-11,2020-04-11,2020-04-10,40s,FEMALE,CC,Resolved,No,2253


In [8]:
caseDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294144 entries, 1 to 294144
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Accurate_Episode_Date  294144 non-null  datetime64[ns]
 1   Case_Reported_Date     294144 non-null  datetime64[ns]
 2   Test_Reported_Date     283613 non-null  datetime64[ns]
 3   Specimen_Date          292320 non-null  datetime64[ns]
 4   Age_Group              294144 non-null  object        
 5   Client_Gender          294144 non-null  object        
 6   Case_AcquisitionInfo   294144 non-null  object        
 7   Outcome1               294144 non-null  object        
 8   Outbreak_Related       294144 non-null  object        
 9   Reporting_PHU_ID       294144 non-null  int64         
dtypes: datetime64[ns](4), int64(1), object(5)
memory usage: 24.7+ MB


In [9]:
print(caseDf.Age_Group.unique())
print(caseDf.Client_Gender.unique())
print(caseDf.Case_AcquisitionInfo.unique())
print(caseDf.Outcome1.unique())
print(caseDf.Outbreak_Related.unique())

['40s' '50s' '60s' '30s' '20s' '70s' '80s' '90+' '<20' 'UNKNOWN']
['FEMALE' 'MALE' 'UNSPECIFIED' 'GENDER DIVERSE']
['OB' 'CC' 'NO KNOWN EPI LINK' 'TRAVEL' 'MISSING INFORMATION'
 'UNSPECIFIED EPI LINK']
['Resolved' 'Fatal' 'Not Resolved']
['Yes' 'No']


4) Create dictionaries for every column you want to convert from an object to an integer.

In [10]:
ageGroup = {'UNKNOWN':0, '<20':1, '20s':2, '30s':3, '40s':4, '50s':5, '60s':6, '70s':7, '80s':8, '90+':9}
clientGender = {'UNSPECIFIED':0, 'FEMALE':1, 'MALE':2, 'GENDER DIVERSE':3}
caseAcquistionInfo = {'MISSING INFORMATION':0, 'CC':1, 'OB':2, 'TRAVEL':3, 'NO KNOWN EPI LINK':4, 'UNSPECIFIED EPI LINK':5}
outcome1 = {'Resolved':0, 'Not Resolved':1, 'Fatal':2}
outbreakRelated = {'No':0, 'Yes':1}

5) Insert these new columns into the caseDf using the dictionaries defined above.

In [11]:
%%time
caseDf['AgeGroupId'] = caseDf.apply(lambda row: ageGroup[row.Age_Group], axis=1)
caseDf['ClientGenderId'] = caseDf.apply(lambda row: clientGender[row.Client_Gender], axis=1)
caseDf['CaseAcquisitionInfoId'] = caseDf.apply(lambda row: caseAcquistionInfo[row.Case_AcquisitionInfo], axis=1)
caseDf['Outcome1Id'] = caseDf.apply(lambda row: outcome1[row.Outcome1], axis=1)
caseDf['OutbreakRelatedId'] = caseDf.apply(lambda row: outbreakRelated[row.Outbreak_Related], axis=1)
caseDf.head(10)

# This only uses 1 cpu ... 
#     mbp ...
#     CPU times: user 1min 12s, sys: 695 ms, total: 1min 13s
#     Wall time: 1min 13s

#     kauwitb ... 
#     CPU times: user 21.7 s, sys: 191 ms, total: 21.9 s
#     Wall time: 21.9 s



CPU times: user 21.1 s, sys: 168 ms, total: 21.3 s
Wall time: 21.3 s


,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,AgeGroupId,ClientGenderId,CaseAcquisitionInfoId,Outcome1Id,OutbreakRelatedId
Row_ID,,,,,,,,,,,,,,,
1,2020-04-07,2020-04-24,2020-04-24,2020-04-22,40s,FEMALE,OB,Resolved,Yes,2253,4,1,2,0,1
2,2020-05-29,2020-06-05,2020-06-05,2020-06-05,50s,FEMALE,OB,Resolved,Yes,2253,5,1,2,0,1
3,2020-03-26,2020-03-30,2020-03-30,2020-03-26,40s,MALE,CC,Resolved,No,2253,4,2,1,0,0
4,2020-05-17,2020-05-18,2020-05-18,2020-05-17,40s,FEMALE,CC,Resolved,No,2253,4,1,1,0,0
5,2020-04-05,2020-04-11,2020-04-11,2020-04-10,40s,FEMALE,CC,Resolved,No,2253,4,1,1,0,0
6,2020-04-23,2020-05-25,2020-05-25,2020-04-24,60s,MALE,CC,Resolved,No,2253,6,2,1,0,0
7,2020-05-16,2020-05-13,2020-05-13,2020-05-12,40s,MALE,NO KNOWN EPI LINK,Fatal,No,2253,4,2,4,2,0
8,2020-06-15,2020-06-16,2020-06-16,2020-06-15,40s,FEMALE,NO KNOWN EPI LINK,Resolved,No,2253,4,1,4,0,0
9,2020-04-11,2020-04-23,2020-04-21,2020-04-21,30s,FEMALE,CC,Resolved,No,2253,3,1,1,0,0


In [12]:
caseDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294144 entries, 1 to 294144
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Accurate_Episode_Date  294144 non-null  datetime64[ns]
 1   Case_Reported_Date     294144 non-null  datetime64[ns]
 2   Test_Reported_Date     283613 non-null  datetime64[ns]
 3   Specimen_Date          292320 non-null  datetime64[ns]
 4   Age_Group              294144 non-null  object        
 5   Client_Gender          294144 non-null  object        
 6   Case_AcquisitionInfo   294144 non-null  object        
 7   Outcome1               294144 non-null  object        
 8   Outbreak_Related       294144 non-null  object        
 9   Reporting_PHU_ID       294144 non-null  int64         
 10  AgeGroupId             294144 non-null  int64         
 11  ClientGenderId         294144 non-null  int64         
 12  CaseAcquisitionInfoId  294144 non-null  int6

6) Drop the columns we no longer need.

In [13]:
caseDf.drop(['Accurate_Episode_Date', 'Test_Reported_Date', 'Specimen_Date', 'Age_Group','Client_Gender','Case_AcquisitionInfo','Outcome1','Outbreak_Related'], axis = 1, inplace=True)
caseDf.head(10)

,Case_Reported_Date,Reporting_PHU_ID,AgeGroupId,ClientGenderId,CaseAcquisitionInfoId,Outcome1Id,OutbreakRelatedId
Row_ID,,,,,,,
1,2020-04-24,2253,4,1,2,0,1
2,2020-06-05,2253,5,1,2,0,1
3,2020-03-30,2253,4,2,1,0,0
4,2020-05-18,2253,4,1,1,0,0
5,2020-04-11,2253,4,1,1,0,0
6,2020-05-25,2253,6,2,1,0,0
7,2020-05-13,2253,4,2,4,2,0
8,2020-06-16,2253,4,1,4,0,0
9,2020-04-23,2253,3,1,1,0,0


In [14]:
caseDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294144 entries, 1 to 294144
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Case_Reported_Date     294144 non-null  datetime64[ns]
 1   Reporting_PHU_ID       294144 non-null  int64         
 2   AgeGroupId             294144 non-null  int64         
 3   ClientGenderId         294144 non-null  int64         
 4   CaseAcquisitionInfoId  294144 non-null  int64         
 5   Outcome1Id             294144 non-null  int64         
 6   OutbreakRelatedId      294144 non-null  int64         
dtypes: datetime64[ns](1), int64(6)
memory usage: 18.0 MB


7) Save the caseDf into a csv file

In [15]:
caseDf.to_csv('PositiveCases.csv')